In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import ta
from ta.volume import VolumeWeightedAveragePrice
from ta.trend import SMAIndicator
from ta.trend import EMAIndicator
from ta.volatility import AverageTrueRange
import time
import datetime
from datetime import date
from datetime import timedelta
from nsepy import get_history
import yfinance as yf

In [2]:
#Getting the list of tickers
df=pd.read_excel('Stock_List.xlsx', header= 1)
Tickers = df["Symbol"]
Tickers = Tickers.astype(str) + ".NS"

In [3]:
#Start Date for the analysis
start_date = "2020-01-01"
today = datetime.datetime.today()

In [4]:
def Add_Indicators(Data):
    EMA20  = EMAIndicator(close = Data["Open"], window = 20, fillna = True)
    EMA30  = EMAIndicator(close = Data["Low"], window = 30, fillna = True)
    EMA50  = EMAIndicator(close = Data["Close"], window = 50, fillna = True)
    EMA100 = EMAIndicator(close = Data["Close"], window = 100, fillna = True)
    EMA200 = EMAIndicator(close = Data["Close"], window = 200, fillna = True)
    ATR    = AverageTrueRange(high = Data["High"], low = Data["Low"], close = Data["Close"], window = 15, fillna = True)
    

    Data["EMA20"]  = EMA20.ema_indicator()
    Data["EMA30"]  = EMA30.ema_indicator()
    Data["EMA50"]  = EMA50.ema_indicator()
    Data["EMA100"] = EMA100.ema_indicator()
    Data["EMA200"] = EMA200.ema_indicator()
    Data["20_Days_EMA_High"] = Data["EMA20"].rolling(min_periods = 1, window = 20, center = False).max()
    Data["20_Days_EMA_Low"] = Data["EMA20"].rolling(min_periods = 1, window = 20, center = False).min()

    EMA20_20 = EMAIndicator(close = Data["EMA20"], window = 20, fillna = True)
    Data["EMA20_10"] = EMA20_20.ema_indicator()

    Data["ATR"] = ATR.average_true_range()

    return Data

In [5]:
#Checking conditions for trade
def check_weekly_conditions(Data_Weekly, j):

    flag = 1
    if(Data_Weekly["Close"][j] >=50):
        
         if(Data_Weekly["Volume"][j-1]>=300000):

            if(((Data_Weekly["Open"][j-1]-Data_Weekly["Close"][j-1]) < (Data_Weekly["Open"][j-2] - Data_Weekly["Close"][j-2]))):

                One_Half_Month = Data_Weekly[j-6:j]
                Avg_Green_Candles =One_Half_Month[One_Half_Month["Close"]>One_Half_Month["Open"]].Close.mean() - One_Half_Month[One_Half_Month["Close"]>One_Half_Month["Open"]].Open.mean()

                Avg_Red_Candles =One_Half_Month[One_Half_Month["Open"]>One_Half_Month["Close"]].Open.mean() - One_Half_Month[One_Half_Month["Open"]>One_Half_Month["Close"]].Close.mean()

                if(Avg_Green_Candles >= 1.1* Avg_Red_Candles):
                    return True, flag

    return False, flag


In [6]:
def check_daily_conditions(Data,j, Small_Cap):
    if(Data["Close"][j] >= 0.8*Data["High"][j-252:j].max()):
            if(Data["Close"][j] >= 1.75*Data["Low"][j-252:j].min()):
                if(Data["EMA50"][j] > Data["EMA100"][j]):
                        if(Data["EMA100"][j] > Data["EMA200"][j]):
                            if(Data["20_Days_EMA_High"][j] - Data["20_Days_EMA_Low"][j] >= 0.02 * Data["Close"][j]):
                                if(Data["EMA50"][j]<= Data["EMA30"][j]):
                                    if(Data["Close"][j]<=3*Data["Low"][j-252:j].min()):

                                        Index = Small_Cap[Small_Cap.index == Data.index[j]]
                                        if(Index.empty == True):
                                            Index = Small_Cap[Small_Cap.index == Data.index[j+1]]
                                        if(Index["EMA50"][0] > Index["EMA100"][0]):
                                            if(Index["Close"][0] > (Index["EMA20"][0])):
                                                
                                                return True
                        
    return False

In [7]:
#Checking Buying Conditions
def buying_conditions(Data,j):
    if(Data["Close"][j] > Data["EMA20"][j]):
        if(Data["Close"][j] - Data["Open"][j] >= 0.02 * Data["Open"][j]):
            if(Data["Close"][j] >= Data["Close"][j-1]):
                if(Data["Close"][j] - Data["EMA30"][j] <= 0.125 * Data["EMA30"][j]):
                    return True

    return False 

In [8]:
#Cheking Exit Conditions
def exit_conditions(Data, k, money_invested, Shares):
    if(Data["Close"][k]*Shares <= (1-0.0875) * money_invested):
        return True, "Loss"
    if(Data["Close"][k]*Shares >= 1.15 * money_invested):
        return True, "Profit"
    
    return False,0

In [9]:
def trailing_exit_conditions(Data, k, money_invested, Shares, trailing_stop_loss):
    
    stop_loss = Data["Close"][k] - 1 * Data["ATR"][k]
    final_stop_loss = max(stop_loss, trailing_stop_loss)

    if(Data["Close"][k] <= final_stop_loss):
        if(Data["Close"][k]*Shares > money_invested):
            return True, "Profit", final_stop_loss
        if(Data["Close"][k]* Shares <= money_invested):
            return True, "Loss", final_stop_loss

    return False,0, final_stop_loss  

In [10]:
%%time

Small_Cap = yf.download("^CRSLDX", start=start_date, interval= "1d", threads =True, group_by = "ticker")
Small_Cap = Small_Cap.drop("Adj Close", axis=1)
Small_Cap = Add_Indicators(Small_Cap)

Trades = pd.DataFrame(columns= ["Day", "Stock"])
for ticker in Tickers:
    print(ticker)
    #Downloading Data
    Data_IntraDay = yf.download(ticker, period="1d", interval= "5m", threads =True, group_by = "ticker")
    I_Date = Data_IntraDay.index.date[0]
    I_Open = Data_IntraDay["Open"][0]
    I_Close = Data_IntraDay["Close"][-1]
    I_High = Data_IntraDay.High.max()
    I_Low = Data_IntraDay.Low.min()
    I_Adj = Data_IntraDay["Adj Close"][-1]
    I_Volume = Data_IntraDay["Volume"].sum()

    Data = yf.download(ticker, start=start_date, period="1d", interval= "1d", threads =True, group_by = "ticker")

    if(Data["Close"][-1] < 50 ):
        continue
    if(Data.shape[0] < 252): #Not considering a stock if it isn't in the market for atleast a year
        continue

    for i in range(6):
        if((I_Date == Data.index[-1].date() + timedelta(days = i) )  and (I_Date!= Data.index[-1].date() )):
            I_Date = Data.index[-1].date() + timedelta(days = i)
            Data = Data.append(pd.DataFrame(index = [I_Date]))
            Data["Open"][-1] = I_Open
            Data["Close"][-1] = I_Close
            Data["High"][-1] = I_High
            Data["Low"][-1] = I_Low
            Data["Adj Close"][-1] = I_Adj
            Data["Volume"][-1] = I_Volume
            break

    Data_Weekly = yf.download(ticker, start=start_date, period="1wk", interval= "1wk", threads =True, group_by = "ticker")
    Data= Data.drop("Adj Close", axis=1)
    Data_Weekly= Data_Weekly.drop("Adj Close", axis=1)
    
    #Adding Indicators 
    Data = Add_Indicators(Data)
    Data_Weekly = Add_Indicators(Data_Weekly)

    j = Data.shape[0] - 1     
    if(check_daily_conditions(Data, j, Small_Cap)==True):

        mask = ((Data_Weekly.index >= Data.index[j]) & (Data_Weekly.index < Data.index[j] + timedelta(days =7)))
        dummy = Data_Weekly[mask]
        for z in range(52, Data_Weekly.shape[0]):
            if(Data_Weekly.index[z] == dummy.index[0]):
                break
        Week_Start = z - 1  

        passed_conditions, flag = check_weekly_conditions(Data_Weekly, Week_Start)

        if(passed_conditions == True):

            #Buying Phase
            if(buying_conditions(Data,j)==True):

                new_row = {"Day": Data.index[-1], "Stock":ticker}
                Trades = Trades.append(new_row, ignore_index = True)




  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
STERTOOLS.NS
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
KARDA.NS
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
MANAKSIA.NS
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
TRIL.NS
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
DBREALTY.NS
[*********************100%****

In [11]:
Trades

,Day,Stock
0,2021-07-15,BLS.NS
1,2021-07-15,STCINDIA.NS


In [33]:
#For trial on a sample of stocks
for ticker in range(395,400):   
    #Downloading Data
    Data_IntraDay = yf.download(Tickers[ticker], period="1d", interval= "5m", threads =True )
    I_Date = Data_IntraDay.index.date[0]
    I_Open = Data_IntraDay["Open"][0]
    I_Close = Data_IntraDay["Close"][-1]
    I_High = Data_IntraDay.High.max()
    I_Low = Data_IntraDay.Low.min()
    I_Adj = Data_IntraDay["Adj Close"][-1]
    I_Volume = Data_IntraDay["Volume"].sum()

    Data = yf.download(Tickers[ticker], start=start_date, period="1d", interval= "1d", threads =True, group_by = "ticker")
    if(Data.shape[0] < 252): #Not considering a stock if it isn't in the market for atleast a year
        continue

    for i in range(6):
        if((I_Date == Data.index[-1].date() + timedelta(days = i) )  and (I_Date!= Data.index[-1].date() )):
            I_Date = Data.index[-1].date() + timedelta(days = i)
            Data = Data.append(pd.DataFrame(index = [I_Date]))
            Data["Open"][-1] = I_Open
            Data["Close"][-1] = I_Close
            Data["High"][-1] = I_High
            Data["Low"][-1] = I_Low
            Data["Adj Close"][-1] = I_Adj
            Data["Volume"][-1] = I_Volume
            break

    Data_Weekly = yf.download(Tickers[ticker], start=start_date, period="1wk", interval= "1wk", threads =True, group_by = "ticker")
    Data= Data.drop("Adj Close", axis=1)
    Data_Weekly= Data_Weekly.drop("Adj Close", axis=1)

    #Adding Indicators 
    Data = Add_Indicators(Data)
    Data_Weekly = Add_Indicators(Data_Weekly)



    j = Data.shape[0] - 1     
    if(check_daily_conditions(Data, j)==True):

        mask = ((Data_Weekly.index >= Data.index[j]) & (Data_Weekly.index < Data.index[j] + timedelta(days =7)))
        dummy = Data_Weekly[mask]
        for z in range(52, Data_Weekly.shape[0]):
            if(Data_Weekly.index[z] == dummy.index[0]):
                break
        Week_Start = z - 1  

        passed_conditions, flag = check_weekly_conditions(Data_Weekly, Week_Start)

        if(passed_conditions == True):

            #Buying Phase
            if(buying_conditions(Data,j)==True):

                new_row = {"Day": Data.index[-1], "Stock":ticker}
                Trades = Trades.append(new_row, ignore_index = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
